<a href="https://colab.research.google.com/github/Yousefalmutairi91/KDM_ICP4/blob/main/KDM_ICP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ICP #4

In [1]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.0 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

Next, we will install Apache Spark 3.0.1 with Hadoop 2.7

In [2]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

Now, we just need to unzip that folder.

In [3]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

There is one last thing that we need to install and that is the findspark library. It will locate Spark on the system and import it as a regular library.

In [4]:
!pip install -q findspark

Now that we have installed all the necessary dependencies in Colab, it is time to set the environment path. This will enable us to run Pyspark in the Colab environment.

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

In [15]:
import findspark
findspark.init()

If you want to know the location where Spark is installed, use findspark.find()

In [7]:
findspark.find()

'/content/spark-3.0.1-bin-hadoop2.7'

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

You can give a name to the session using appName() and add some configurations with config() if you wish.

In [28]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Finally, print the SparkSession variable.

In [9]:
spark

In [11]:
#we need to load the dataset. We will use the read.csv module. 
#The inferSchema parameter provided will enable Spark to automatically determine the data type for each column but it has to go over the data once.
# If you don’t want that to happen, then you can instead provide the schema explicitly in the schema parameter.

df = spark.read.csv("/content/data.csv", header=True, inferSchema= True)
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [12]:
#If you didn't set inderShema to True, here is what is happening to the type. There are all in string.
df_string = spark.read.csv("/content/data.csv", header=True, inferSchema=  False)
df_string.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



# 1) Perform at least 3 transformations on the data.csv file.

---



In [30]:
file = sc.textFile("/content/data.csv").map(lambda line : line.split(","))

***#1 transformation***

In [79]:
#Filter function : Return a new dataset formed by selecting those elements of the source on which func returns true.
#ex: customer ID ; 7795-CFOCW


data1 = file.filter(lambda line: "One year" in line).count()
print(data1)

1473


***#2 transformation***

In [45]:
#intersect two column and print out all female who pays via Electronic check

f1= df.filter(df['gender'] == 'Female')
f2= df.filter(df['PaymentMethod'] == 'Electronic check')
f3 = f1.intersect(f2).show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|   PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+
|9880-TDQAC|Female|            0|    Yes|       Yes|    72|          No|No phone service|            DSL|            No|         Yes|             Yes|        Yes|        Yes|    

In [77]:
# MAP function = applies transformation function on dataset and returns same number of elements in distributed dataset.

fm = file.map(lambda column : (column [20], 1)).reduceByKey(lambda x, y: x + y)

In [76]:
fm.collect()

[('Churn', 1), ('No', 5174), ('Yes', 1869)]

# 2) Perform3 actions on the data.csv file.

---



***Action #1***

In [49]:
#count the number of elements

df.count()

7043

In [51]:
#counts by gender and contract to show us for example who prefer one or two years contract...

counts1 = df.groupBy("gender","contract").count()
counts1.show()

+------+--------------+-----+
|gender|      contract|count|
+------+--------------+-----+
|Female|      One year|  718|
|  Male|      One year|  755|
|  Male|Month-to-month| 1950|
|Female|Month-to-month| 1925|
|  Male|      Two year|  850|
|Female|      Two year|  845|
+------+--------------+-----+



***Action #2***

In [123]:
#Collect all CustomerID and Monthely charges... 

dataCollect = df.select("customerID","MonthlyCharges").collect()
print (dataCollect)

[Row(customerID='7590-VHVEG', MonthlyCharges=29.85), Row(customerID='5575-GNVDE', MonthlyCharges=56.95), Row(customerID='3668-QPYBK', MonthlyCharges=53.85), Row(customerID='7795-CFOCW', MonthlyCharges=42.3), Row(customerID='9237-HQITU', MonthlyCharges=70.7), Row(customerID='9305-CDSKC', MonthlyCharges=99.65), Row(customerID='1452-KIOVK', MonthlyCharges=89.1), Row(customerID='6713-OKOMC', MonthlyCharges=29.75), Row(customerID='7892-POOKP', MonthlyCharges=104.8), Row(customerID='6388-TABGU', MonthlyCharges=56.15), Row(customerID='9763-GRSKD', MonthlyCharges=49.95), Row(customerID='7469-LKBCI', MonthlyCharges=18.95), Row(customerID='8091-TTVAX', MonthlyCharges=100.35), Row(customerID='0280-XJGEX', MonthlyCharges=103.7), Row(customerID='5129-JLPIS', MonthlyCharges=105.5), Row(customerID='3655-SNQYZ', MonthlyCharges=113.25), Row(customerID='8191-XWSZG', MonthlyCharges=20.65), Row(customerID='9959-WOFKT', MonthlyCharges=106.7), Row(customerID='4190-MFLUW', MonthlyCharges=55.2), Row(customerI

***Action #3***

In [129]:
#Take top 3 CustomerID and Total charges... 

#df.take(3)

DataTake = df.select("customerID","TotalCharges").take(3)
print (DataTake)

[Row(customerID='7590-VHVEG', TotalCharges='29.85'), Row(customerID='5575-GNVDE', TotalCharges='1889.5'), Row(customerID='3668-QPYBK', TotalCharges='108.15')]
